In [2]:
import utils
import numpy as np
import pynq, time
import matplotlib.pyplot as plt

In [3]:
bitfile ='../bitfiles/signal_generator/signal_generator.bit'
reg_start = 0x43c00000
reg_len = 2**3*4
#reg[0]    = bit0:en read
#            bit1:rst_read
#            bit2:continous
#reg[4:8]  = dec_rate
#reg[8:12] = dac_default 

bram_start = 0x43c08000
bram_len = 2**13*4

In [4]:
pynq.Overlay(bitfile)    

In [5]:
reg = utils.mmio(reg_start,reg_len)
bram = utils.mmio(bram_start, bram_len)

In [30]:
def upload_bram_data(bram, reg, dac0, dac1,dec):
    """ dac0 and dac1 should be less than 1
    """
    reg.write(0, np.array(0b010))
    dat = np.vstack([dac0,dac1])
    dat = dat.T.flatten()
    dat = utils.float2fixed(dat, 14,13)
    bram.write(0, dat, 'h')
    reg.write(4,np.array(dec))
    
    
def dac_sinewave(bram, reg, f1=5, f2=2, dec=0, offset=0, amp=0.5, fs=250):
    """ The amplitude and offset must be such that the signal is less than 1
        dec: decimation
    """
    total = 8192
    t = np.arange(8192)
    f1_fixed = int(f1/fs*total)
    f2_fixed = int(f1/fs*total)
    x1 = amp*np.sin(2*np.pi*t*f1_fixed/total)+offset
    print(f1_fixed)
    x2 = amp*np.sin(2*np.pi*t*f2_fixed/total)+offset
    upload_bram_data(bram, reg, x1, x2,dec)
    reg.write(0, np.array(0b101))
    
    
    

In [42]:
dac_sinewave(bram, reg,f1=1,f2=2,dec=1, amp=0.5, offset=0.0, fs=250)
##I measure half of the actual frequency... could be becouse even with dec =0 it needs one cycle...
##or there is a problem with the clock generated by the PLL....

32


In [16]:
def cont_transfer(reg, dec=0):
    """ The data in the bram is continously output by the DAC
    """
    reg.write(0, np.array(0b010))
    reg.write(4, np.array(dec), 'u4')
    reg.write(0, np.array(0b000))
    reg.write(0, np.array(0b101))

def beat_transfer(reg_mm, dec=0):
    reg.write(0, np.array(0b010))
    reg.write(4, np.array(dec), 'u4')
    reg.write(0, np.array(0b000))
    reg.write(0, np.array(0b1))

def upload_beat(bram, reg, dat1,dat2, dfl_val1,dfl_val2, dec=0):
    """ dat1, dat2: data for the dac
        dfl_val1, dfl_val2: are the values that the dac takes when finish the beat
        dec: decimation
    """
    reg.write(0, np.array(0b010))
    reg.write(0, np.array(0b000))
    dat = np.vstack([dfl_val1,dfl_val2])
    dat = dat.T.flatten()
    dat = utils.float2fixed(dat, 14,13)
    reg.write(8, dat, 'i2')
    upload_bram_data(bram, reg, dat0, dat1, dec)


In [ ]:
dat1 = np.linspace(-0.98,0.98,8192,endpoint=False)
dat2 = -dat1
dfl_val1 = 0.98
dfl_val2 = 0.5

#upload_beat(bram, reg, dat1, dat2, dfl_val1,dfl_val2)
#cont_transfer(reg, dec=0)
#beat_transfer(reg, dec=2**15)
#